In [15]:
import numpy as np
import random
import math

# Game Engine

In [16]:
class Game():
    def __init__(self):
        self.board = np.zeros([4, 4])
        self.last = None

    def new_piece(self):
        flattened_board = self.board.reshape(16)
        empty_spots = np.where(flattened_board == 0)[0]
        
        if len(empty_spots) == 0:
            return False
        else:
            new_spot = empty_spots[random.randint(0, len(empty_spots) - 1)]
            print(f"New cell created at {math.trunc(new_spot / 4)},{new_spot % 4}")
            flattened_board[new_spot] = 2
            self.board = flattened_board.reshape(4, 4)

        return True

    def has_lost(self):
        # TODO
        return False

    def has_won(self):
        flattened_board = self.board.reshape(16)        
        return len(np.where(flattened_board == 2048)[0]) > 0
    
    def cycle(self):
        self.new_piece()

    def left(self):    
        if self.last != 'L':
            for m in range(0, 4):
                merged = False # only allow one merge per row per turn, as per the rules
                
                for i in range(0, 4):
                    for n in range(1, 4):
                        if not self.board[m][n] == 0 and not merged and self.board[m][n - 1] == self.board[m][n]:
                            print(f"Cells merged into {m},{n - 1}")
                            self.board[m][n - 1], self.board[m][n] = self.board[m][n - 1] * 2, 0
                            merged = True
                            
                        if self.board[m][n - 1] == 0 and self.board[m][n] != 0:
                            self.board[m][n - 1], self.board[m][n] = self.board[m][n], 0
                            
            self.last = 'L'
            self.cycle()

    def right(self):
        if self.last != 'R':        
            for m in range(0, 4):
                merged = False # only allow one merge per row per turn, as per the rules
    
                for i in range(0, 4):
                    for n in range(2, -1, -1):
                        if not self.board[m][n] == 0 and not merged and self.board[m][n] == self.board[m][n + 1]:
                            print(f"Cells merged into {m},{n + 1}")
                            self.board[m][n], self.board[m][n + 1] = 0, self.board[m][n] * 2
                            merged = True
                        
                        if self.board[m][n] != 0 and self.board[m][n + 1] == 0:
                            self.board[m][n], self.board[m][n + 1] = 0, self.board[m][n]

            self.last = 'R'
            self.cycle()
            
    def up(self):
        if self.last != 'U':        
            for n in range(0, 4):
                merged = False # only allow one merge per column per turn, as per the rules
                
                for i in range(0, 4):
                    for m in range(1, 4):
                        if not self.board[m][n] == 0 and not merged and self.board[m - 1][n] == self.board[m][n]:
                            print(f"Cells merged into {m - 1},{n}")
                            self.board[m - 1][n], self.board[m][n] = self.board[m - 1][n] * 2, 0
                            merged = True
                            
                        if self.board[m - 1][n] == 0 and self.board[m][n] != 0:
                            self.board[m - 1][n], self.board[m][n] = self.board[m][n], 0

            self.last = 'U'
            self.cycle()

    def down(self):
        if self.last != 'D':        
            for n in range(0, 4):
                merged = False # only allow one merge per column per turn, as per the rules
    
                for i in range(0, 4):
                    for m in range(2, -1, -1):
                        if not self.board[m][n] == 0 and not merged and self.board[m][n] == self.board[m + 1][n]:
                            print(f"Cells merged into {m + 1},{n}")
                            self.board[m][n], self.board[m + 1][n] = 0, self.board[m][n] * 2
                            merged = True
                        
                        if self.board[m][n] != 0 and self.board[m + 1][n] == 0:
                            self.board[m][n], self.board[m + 1][n] = 0, self.board[m][n]

            self.last = 'D'
            self.cycle()

# Test new piece generation and basic movement

In [17]:
game = Game()
game.board

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [18]:
game.new_piece()
game.new_piece()
game.board

New cell created at 3,0
New cell created at 0,1


array([[0., 2., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [2., 0., 0., 0.]])

In [19]:
game.left()
game.board

New cell created at 1,2


array([[2., 0., 0., 0.],
       [0., 0., 2., 0.],
       [0., 0., 0., 0.],
       [2., 0., 0., 0.]])

In [20]:
out = game.right()
out, game.board 

New cell created at 3,1


(None,
 array([[0., 0., 0., 2.],
        [0., 0., 0., 2.],
        [0., 0., 0., 0.],
        [0., 2., 0., 2.]]))

# Test merge left

In [21]:
game = Game()
game.board[0][0], game.board[0][3] = 2, 2
game.board

array([[2., 0., 0., 2.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [22]:
game.left()
game.board

Cells merged into 0,0
New cell created at 2,2


array([[4., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 2., 0.],
       [0., 0., 0., 0.]])

# Test merge right

In [23]:
game = Game()
game.board[0][0], game.board[0][3] = 2, 2
game.board

array([[2., 0., 0., 2.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [24]:
game.right()
game.board

Cells merged into 0,3
New cell created at 1,2


array([[0., 0., 0., 4.],
       [0., 0., 2., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

# Test merge up

In [25]:
game = Game()
game.board[0][0], game.board[3][0] = 2, 2
game.board

array([[2., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [2., 0., 0., 0.]])

In [26]:
game.up()
game.board

Cells merged into 0,0
New cell created at 3,0


array([[4., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [2., 0., 0., 0.]])

# Test merge down

In [27]:
game = Game()
game.board[0][0], game.board[3][0] = 2, 2
game.board

array([[2., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [2., 0., 0., 0.]])

In [28]:
game.down()
game.board

Cells merged into 3,0
New cell created at 2,3


array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 2.],
       [4., 0., 0., 0.]])